## Import required libraries and Load the data

In [ ]:
!pip install ipython-autotime --quiet

In [ ]:
%load_ext autotime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import xarray as xr

import warnings
warnings.filterwarnings('ignore')

time: 2.06 s (started: 2022-07-26 05:13:02 +00:00)


In [ ]:
data = xr.open_dataset('/conten')

In [ ]:
%load_ext google.colab.data_table
data = pd.read_csv('/content/output_1.csv') #address to your data file.
data.head()

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data.index = data['date']
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.info()

## Exploratory data analysis

In [ ]:
data.describe()

In [ ]:
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#Customize the graphs or plots
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 10
matplotlib.rcParams['figure.figsize'] = (15,4)
matplotlib.rcParams['figure.facecolor'] = '#0000000F'

## Splitting the data 

In [ ]:
data['Year'] = data['date'].dt.year

In [ ]:
data.head()

In [ ]:
plt.title('No. of Observations per Year')
sns.countplot(x = data.Year);

In [ ]:
data.drop(['Year','date',],axis=1,inplace=True)
data.head()

In [ ]:
data.isna().sum()

In [ ]:
data = data.replace(0,np.nan)
data.isna().sum()

In [ ]:
train = data[0:int(data.shape[0]*0.8)].copy()
train.tail()

In [ ]:
test = data[int(data.shape[0]*0.8):].copy()
test.head()

In [ ]:
print(train.shape)
print(test.shape)

## Data Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(train)
train = scaler.transform(train)

## Dividing the data into timeshifts

In [ ]:
#Create two empty lists x_train and y_train
X_train = []
y_train = []

In [ ]:
n = train.shape[1] // 7 # step to find number of locations dynamically.
target_list = []
for i in range(n):
  index_no = data.columns.get_loc('t2m_{}'.format(i))
  target_list.append(index_no)

In [ ]:
for i in range(7, train.shape[0]):
  X_train.append(train[i-7:i])
  y_train.append(train[i,target_list])

In [ ]:
#Converting Lists into numpy arrays
X_train = np.array(X_train) 
y_train =  np.array(y_train)

In [ ]:
X_train.shape, y_train.shape

## Build the Model

In [ ]:
np.random.seed(1337)

In [ ]:
#Import libraries
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout

In [ ]:
X_train.shape[1],X_train.shape[2]

In [ ]:
model = Sequential()
model.add(LSTM(100 , return_sequences= True , activation='relu', input_shape = (X_train.shape[1],X_train.shape[2]) , kernel_regularizer= keras.regularizers.l2(0.001))) 
model.add(Dropout(0.3)) # Add a dropout layer in order to avoid overfitting.
model.add(LSTM(200, return_sequences= True)) # Hidden layer,Another Lstm layer for stacked Lstm method.
model.add(Dropout(0.3))
model.add(LSTM(100))
# final Output layer.
model.add(Dense(y_train.shape[1]))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adamax', loss='mse')

In [ ]:
# fitting the model with train data
history = model.fit(X_train, y_train , epochs=25, batch_size=128)

In [ ]:
# plotting the loss value of each epoch
plt.figure(figsize=(8,8))
plt.title('Loss Function')
plt.xlabel('No.of epochs')
plt.ylabel('rmse')
plt.plot(history.history['loss'])

## Evaluate the Model

In [ ]:
train_preds = model.predict(X_train)

In [ ]:
train_preds.shape

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
RMSE = np.sqrt(mean_squared_error(y_train, train_preds))
print(round(RMSE,3))

In [ ]:
from sklearn.metrics import r2_score
Score = r2_score(y_train, train_preds)
print(round(Score,2)*100)

In [ ]:
print(y_train[:,1])

In [ ]:
# plotting the original Vs Predicted values graphically.
plt.figure(figsize=(30,8))
plt.title('Original vs predicted(optimizer = adamax)')
plt.xlabel('Over the data')
plt.ylabel('Temperature(air_temp)')
plt.plot(y_train)
plt.plot(train_preds)
plt.legend(['Original','Predicted'])
plt.show()

In [ ]:
plt.title('Scatter plot (Test Preds vs Original)')
plt.xlabel('Original Values')
plt.ylabel('Predicted Values')
plt.scatter(y_train, train_preds, alpha=0.5,c=y_train,cmap='nipy_spectral',s=y_train)
m = 1
plt.plot(y_train, m*y_train)
plt.show()

## Preprocessing test data

In [ ]:
last_7_days = data[:int(data.shape[0]*0.8)].tail(7).copy()
last_7_days

In [ ]:
test.head()

In [ ]:
#Adding last 30 days of train data and test data.
test_final = last_7_days.append(test)
test_final.tail()

In [ ]:
#Scaling columns in the final test data.
test_final = scaler.transform(test_final)
test_final

In [ ]:
#Creating two empty lists X_test and y_test
X_test = []
y_test = []

In [ ]:
for i in range(7, test_final.shape[0]):
  X_test.append(test_final[i-7:i])
  y_test.append(test_final[i,target_list])

In [ ]:
# converting into array.
X_test , y_test = np.array(X_test) , np.array(y_test)

In [ ]:
X_test.shape , y_test.shape

## Making prediction on test data

In [ ]:
y_preds = model.predict(X_test)

In [ ]:
y_preds.shape

## Evaluating 

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_preds))
print(round(rmse,2))

In [ ]:
r2_score = r2_score(y_test, y_preds)
print(round(r2_score,2)*100)

In [ ]:
# plotting the original Vs Predicted values graphically.
plt.figure(figsize=(15,6))
plt.rcParams.update({'font.size': 9})
plt.title('Original vs predicted(optimizer = adamax)')
plt.xlabel('Over the data')
plt.ylabel('Temperature(air_temp)')
plt.plot(y_test)
plt.plot(y_preds)
plt.legend(['Original','Predicted'])
plt.show()

In [ ]:
plt.title('Scatter plot (Test Preds vs Original)')
plt.xlabel('Original Values')
plt.ylabel('Predicted Values')
plt.scatter(y_test, y_preds, alpha=0.5,c=y_test,cmap='nipy_spectral',s=y_test)
m = 1
plt.plot(y_test, m*y_test)
plt.show()